How to convert a Google sheet to a static marketing website

templates google doc?

templates google sheet?

In [ ]:
var fs = require('fs');
var path = require('path');
var util = require('util');

var GoogleSpreadsheet = require('google-spreadsheet');

function getTemplates(doc) {
    if(typeof doc === 'string') {
        doc = new GoogleSpreadsheet(doc);
    }
    return util.promisify(doc.getInfo)()
        .then(info => {
            console.log('Loaded doc: ' + info.title + ' by ' + info.author.email);
            return info;
        })
        .then(s => s.worksheets.reduce((obj, w) => {
            var name = w.title.toLowerCase().replace(/\s*layout|\s*data|\s*template/ig, '');
            if( typeof obj[name] == 'undefined' ) obj[name] = {template: null, data: null};
            if( w.title.toLowerCase().includes('data') ) obj[name].data = w;
            else obj[name].template = w;
            return obj;
        }, {}))
}
module.exports = getTemplates;


test templates google sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get templates from sheet', () => {
    
    it('should have at least one page', () => {
        return getTemplates(docsId)
            .then(t => {
                console.log(JSON.stringify(Object.keys(t).reduce((obj, cur) => {
                    obj[cur] = {};
                    obj[cur].template = ((t[cur].template || {}).title || '').substr(0, 1000);
                    obj[cur].data = (t[cur].data ? ('yes: ' + t[cur].data.rowCount) : 'no');
                    return obj;
                }, {}), null, 4))
                assert(Object.keys(t).length > 0, 'should have a page called template');
            })
    })
    
    
})


get google data sheet?

google sheet array objects?


In [ ]:

function getDataSheet(worksheet) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            var columns = [];
            return cells.reduce((acc, o, arr) => {
                if (o.row == 1) {
                    columns[o.col] = o.value;
                    return acc;
                }
                if (!acc[o.row - 2]) {
                    acc[o.row - 2] = {};
                }
                acc[o.row - 2][columns[o.col]] = md.render(o.value);
                return acc;
            }, []);
        })
}
module.exports = getDataSheet;


test google data sheet?


google sheet layout template?

Convert a google sheet page to an HTML template



In [ ]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true});

function getLayoutSheet(docInfo, sheetName) {
    var sheet = docInfo.worksheets
        .filter(w => w.title.toLowerCase().includes(sheetName)
            && w.title.toLowerCase().includes('layout'))[0];
    return util.promisify(sheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => htmlSheetMarkup(sheetName, cells))
}

function htmlSheetMarkup(sheetName, cells) {
    const reserved = ['class', 'layout', 'props', 'iterate'];
    var name = sheetName.split(/\s/ig).join('').replace(/layout/ig, '');
    var className = '';
    var imports = [];
    var html = cells.filter((c, i, arr) => c.value)
        .map((c, i, arr) => {
            var result;
            if (reserved.includes(c.value)) {
                var value = arr
                    .filter(c2 => c2.row == c.row && c2.col == c.col + 1)[0].value
                if (c.value == 'layout') {
                    var layout = value.split(/\s/ig).join('-');
                    imports[imports.length] = value;
                    return `
<${layout} v-model="${name}"></${layout}>`
                } else if (c.value == 'iterate') {
                    var layout = value.split(/\s/ig).join('-');
                    imports[imports.length] = value;
                    return `
<div v-for="item in value"><${layout} :value="item"></${layout}></div>`
                } else if (c.value == 'class') {
                    className = value;
                    return '';
                }
            } else if (c.col > 1 && reserved.includes((arr
                .filter(c2 => c2.row == c.row && c2.col == c.col - 1)[0] || {}).value)) {
                return '';
            } else if ((result = (/\{+\s*(value\.)*(.*?)\s*\}+/ig).exec(c.value))) {
                return `
    <div class="${result[2]}">${md.render(c.value)}</div>`;
            }
            return md.render(c.value);
        }).join('        ');
    return { html, className, imports };
}

module.exports = {
    getLayoutSheet,
    htmlSheetMarkup
};

test google sheet template?


output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:

function writeTemplate(className, html, data, sheetName, imports) {
    var formal = sheetName.split(/\s/ig)
        .map(t => t[0].toUpperCase() + t.substr(1))
        .join('')
        .replace(/\s*layout\s*/ig, '');
    var components = imports.map(i => i.split(/\s/ig)
        .map(t => t[0].toUpperCase() + t.substr(1))
        .join('')
        .replace(/\s*layout\s*/ig, ''));
    var importString = components.map(importFormal => 
        `import ${importFormal} from "./${importFormal}.vue";`
    ).join('\n');
    var name = sheetName.replace(/[^a-z]/ig, '');
    fs.writeFileSync(path.join(__dirname, '../src/views/generated/' + formal + '.vue'), `
<template>
    <div class="${formal.toLocaleLowerCase()} ${className}">
${html}
    </div>
</template>

<script>
${importString}

export default {
    name: '${name}',
    props: ['value'],
    components: { ${components.join(',')} },
    data() {
        return { 
            "${name}": JSON.parse(${JSON.stringify(JSON.stringify(data))})
        };
    },
};
</script>
`);
}

module.exports = writeTemplate;


test output sheet template?
